In [122]:
import pandas as pd
from collections import Counter
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import normalize
import requests
from IPython.display import HTML

In [123]:
card_images_dict = eval(requests.get('https://us-central1-royaleapp.cloudfunctions.net/getcardimages').text)

# Read in GC Data

In [57]:
df = pd.read_csv('../../../gc-data-2020.04-100000.csv')

for i in range(1,9):
    df['card'+str(i)] = df['card'+str(i)].apply(lambda x: x.replace('-', ' ').title().replace('Pekka', 'P.E.K.K.A').replace('X Bow', 'X-Bow'))
    
print(df.shape)
df.head()

(100001, 9)


,doc_id,card1,card2,card3,card4,card5,card6,card7,card8
0,2VVQ2RGC8-1585715128000-CYJGP8CL-R,Baby Dragon,Barbarian Barrel,Freeze,Goblins,Graveyard,Ice Wizard,Mega Knight,Tornado
1,RL2QYVY8-1586229529000-92CL822V,Balloon,Barbarian Barrel,Bats,Giant Snowball,Goblins,Ice Spirit,Lumberjack,Princess
2,8RUUL28QR-1585861468000-L2CUU2Y29-R,Baby Dragon,Barbarian Barrel,Firecracker,Golem,Lightning,Lumberjack,Night Witch,Tornado
3,GYRYCPV8-1586233796000-998GRLRY-R,Baby Dragon,Barbarian Barrel,Bomb Tower,Graveyard,Knight,Musketeer,Poison,Skeletons
4,8P8CV9LQC-1586291071000-9LR0C8VU-R,Balloon,Giant Snowball,Ice Golem,Inferno Tower,Miner,Musketeer,Skeletons,Valkyrie


In [35]:
# Out of 100,000 decks, only 903 are used at least 10 times
# How are these decks selected?
decks_str = df[['card'+str(i) for i in range(1,9)]].apply(lambda x: str(sorted(x)), axis=1).values.tolist()

len([(d,c) for d,c in Counter(decks_str).most_common() if c>=10])

903

In [90]:
# Only take unique decks - 15,380 of them
decks = df[['card'+str(i) for i in range(1,9)]].values.tolist()
decks = set([str(sorted(d)) for d in decks if 'Heal' not in d])
decks = [eval(d) for d in decks]
len(decks)

15380

# Read in Excel Stats

In [100]:
card_stats = pd.read_excel('../../../card_stats.xlsx')
stats_dict = dict(zip(card_stats['name'], card_stats.values[:,1:]))
card2ind = dict(zip(card_stats['name'], card_stats.index))

# Create Deck Matrices

In [181]:
percentile_max = 105
percentile_step = 5

In [182]:
deck_matrix = np.array([normalize(np.percentile(np.array([stats_dict[c] for c in deck]), list(np.arange(0,percentile_max,percentile_step)), axis=0).flatten().reshape(-1, 1), norm='l2', axis=0).reshape(1,-1)[0] for deck in tqdm(decks)])

100%|██████████████████████████████████████████████████████████████████████████| 15380/15380 [00:15<00:00, 1001.30it/s]


In [102]:
one_hot_deck_matrix = []
for deck in tqdm(decks):
    blank_vec = np.zeros(len(card_stats['name']))
    for card in deck:
        blank_vec[card2ind[card]] = 1
    blank_vec = normalize(blank_vec.reshape(-1, 1), norm='l2', axis=0).reshape(1,-1)[0]
    one_hot_deck_matrix.append(blank_vec)
    
one_hot_deck_matrix = np.array(one_hot_deck_matrix)

100%|██████████████████████████████████████████████████████████████████████████| 15380/15380 [00:03<00:00, 4514.01it/s]


# Get Recommendations

In [178]:
def display_decks(decks, input_deck):
    deck_image_str = '<table>'
    for deck in decks:
        deck_image_str += '<tr>'
        card_count = 0
        for card in deck:
            if card in input_deck:
                card_count += 1
            deck_image_str += '<td><img src={} style="height:75px"></td>'.format(card_images_dict[card])
        deck_image_str += '<td>{}/8 same cards</td>'.format(card_count)
        deck_image_str += '</tr>'
    deck_image_str += '</table>'
    display(HTML(deck_image_str))

In [202]:
# input_deck = ['Arrows', 'Balloon', 'Guards', 'Lava Hound', 'Mega Minion', 'Miner', 'Minions', 'Zap']
# input_deck = ['Archers', 'Fireball', 'Ice Golem', 'Ice Spirit', 'The Log', 'Skeletons', 'Tesla', 'X-Bow']
# input_deck = ['Giant Snowball', 'Bandit', 'Ram Rider', 'Electro Wizard', 'Barbarian Barrel', 'Poison', 'Baby Dragon', 'P.E.K.K.A']
input_deck = ['Bats', 'Arrows', 'Miner', 'Wall Breakers', 'Musketeer', 'Electro Wizard', 'Mega Knight', 'Bandit']
# input_deck = []
random_ind = np.random.choice(range(len(decks))) #11142

n_recs = 6
input_deck = sorted(input_deck)

In [203]:
if not input_deck:
    input_deck = decks[random_ind]

stat_embedding = normalize(np.percentile(np.array([stats_dict[c] for c in input_deck]), list(np.arange(0,percentile_max,percentile_step)), axis=0).flatten().reshape(-1, 1), norm='l2', axis=0).reshape(1,-1)[0]

onehot_embedding = np.zeros(len(card_stats['name']))
for card in input_deck:
    onehot_embedding[card2ind[card]] = 1
onehot_embedding = normalize(onehot_embedding.reshape(-1, 1), norm='l2', axis=0).reshape(1,-1)[0]

rec_sims = stat_embedding.dot(deck_matrix.T)
onehot_rec_sims = onehot_embedding.dot(one_hot_deck_matrix.T)
blended_sims = rec_sims*.5 + onehot_rec_sims*.5

rec_inds = np.argsort(rec_sims)[::-1][:n_recs]
onehot_rec_inds = np.argsort(onehot_rec_sims)[::-1][:n_recs]
blended_rec_inds = np.argsort(blended_sims)[::-1][:n_recs]

print('Input Deck:')
display_decks([input_deck], input_deck)

print('\nStats Recommendations:')
display_decks([decks[r] for r in rec_inds if decks[r] != input_deck], input_deck)
    
print('\nOne-Hot Recommendations:')
display_decks([decks[r] for r in onehot_rec_inds if decks[r] != input_deck], input_deck)

print('\n 50/50 Blended Recommendations:')
display_decks([decks[r] for r in blended_rec_inds if decks[r] != input_deck], input_deck)

Input Deck:


,,,,,,,,8/8 same cards



Stats Recommendations:


,,,,,,,,7/8 same cards
,,,,,,,,7/8 same cards
,,,,,,,,5/8 same cards
,,,,,,,,6/8 same cards
,,,,,,,,6/8 same cards
,,,,,,,,5/8 same cards



One-Hot Recommendations:


,,,,,,,,7/8 same cards
,,,,,,,,7/8 same cards
,,,,,,,,7/8 same cards
,,,,,,,,6/8 same cards
,,,,,,,,6/8 same cards
,,,,,,,,6/8 same cards



 50/50 Blended Recommendations:


,,,,,,,,7/8 same cards
,,,,,,,,7/8 same cards
,,,,,,,,7/8 same cards
,,,,,,,,6/8 same cards
,,,,,,,,6/8 same cards
,,,,,,,,6/8 same cards
